In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

# Transferència de coneixement

L'objectiu d'avui és aprendre com podem emprar arquitectures ja existents per resoldre els nostres problemes. 

Com objectius secundaris tenim:

1. Conèixer un nou conjunt de dades
2. Entendre en profunditat com és una de les arquitecures més famoses.
3. Guardar i carregar xarxes neuronals

## Dades

El conjunt de dades [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) consta de 60.000 imatges en color de 32x32 pixels etiquetades en 10 classes, amb 6.000 imatges per classe. Hi ha 50.000 imatges d'entrenament i 10.000 imatges de _test_.

### Feina a fer:

1. Adaptar la mateixa xarxa que vareu desenvolupar la setmana anterior per emprar aquest conjunt de dades. `Grayscale` és una funció que transforma imatges a escala de grisos, la podem emprar dins la nostra composició de transformacions.

Si voleu normalitzar les dades, a continuació teniu els valors ja calculats:

  - mitjana: (0.4914, 0.4822, 0.4465)
  - desviació típica: (0.247, 0.243, 0.261)

Una altra funció que pot ser útil és `Resize(mida_desti)` que rep un enter com a paràmetre (la mida final).


In [ ]:
etiquetes = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck",
}

train_batch_size = 64
test_batch_size = 100

# Definim una seqüència (composició) de transformacions 
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256)
    # transforms.Normalize((0.4914, 0.4822, 0.4465) , (0.247, 0.243, 0.261))
    ])

# Descarregam un dataset ja integrat en la llibreria Pytorch
train = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
test = datasets.CIFAR10('../data', train=False, transform=transform)

# Transformam les dades en l'estructura necessaria per entrenar una xarxa
train_loader = torch.utils.data.DataLoader(train, train_batch_size)
test_loader = torch.utils.data.DataLoader(test, test_batch_size)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data


## Transfer learning (Definició de la xarxa)

En aquesta pràctica aplicarem la tècnica de _transfer learning_ a partir d'una de les xarxes més conegudes en el camp de visió per computador: [**AlexNet**](https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf). (ImageNet Classification with Deep Convolutional Neural Network, 2012).

_Pytorch_ ens permet emprar les xarxes més conegudes de manera molt senzilla. [Més informació](https://pytorch.org/vision/stable/models.html).

Per xarxes no tan conegudes podem guardar i carregar els models de manera molt senzilla: [Saving and Loading Models](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

Anem a descarregar-la i a analitzar-la. En aquest cas no només ens baixam la seva arquitectura, també els pesos resultants de l'entrenament.

In [ ]:
alex = models.alexnet(weights=True)

print("-"*50)
print("Arquitectura AlexNet")
print("-"*50)
print(alex)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

--------------------------------------------------
Arquitectura AlexNet
--------------------------------------------------
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classi

Hi ha diverses maneres de realitzar aquesta tècnica les dues més conegudes són:

 - **"Congelar"** els pesos de la part d'extracció de característiques i crear un nou classificador. Això implica que només entrenam una part de la xarxa.
 - **Reentrenar tota la xarxa**.

 Per tal d'evitar el reentrenament necessitam canviar el valor de l'atribut  `requires_grad` al valor `False`. Aquest atribut és propietat de cada tensor. Podem recorrer els tensors mitjançant el següent codi:
 ```
for param in alex.features.parameters():
    param.requires_grad = False
 ```

 ### Feina a fer:

 1. Carregar la xarxa AlexNet i seleccionar la part d'extracció de característiques.
 2. Definir un entorn seqüencial on implementarem el classificador de la xarxa.
 3. Realitzar un entrenament i comparar els resultats amb el primer entrenament (xarxa pròpia): comparar rendiment (accuracy) però també temps dedicat a entrenar i nombre de paràmetres.
 4. Provar de guardar la vostra xarxa i tornar-la a carregar. Per classificar una imatge del conjunt de test.

In [ ]:
# Congelar
for param in alex.features.parameters():
 param.requires_grad = False

my_net =  nn.Sequential(alex.features,
                        nn.Flatten(1,-1),
                        nn.Linear(12544,1568),
                        nn.Linear(1568, 196),
                        nn.Linear(196, 10)
                        )

## Entrenament

[shhht](https://github.com/tqdm/tqdm) si voleu canviar el resum de l'entrenament per una barra de progrés

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target, reduction='mean') 
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item()/ len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='mean') 
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
  
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [ ]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = 6#¿?
lr = 0.00001#¿?

model = my_net.to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # !!!

print("Parameters ", pytorch_total_params)
optimizer = optim.Adam(model.parameters(), lr=lr) #¿?

# Guardam el valor de pèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)


Parameters  19980054
Train Epoch: 0 [0/50000 (0%)]	Loss: 2.299422, Average: 0.035928
Train Epoch: 0 [6400/50000 (13%)]	Loss: 1.641049, Average: 0.025641
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.172119, Average: 0.018314
Train Epoch: 0 [19200/50000 (38%)]	Loss: 1.243204, Average: 0.019425
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.146371, Average: 0.017912
Train Epoch: 0 [32000/50000 (64%)]	Loss: 1.213682, Average: 0.018964
Train Epoch: 0 [38400/50000 (77%)]	Loss: 0.919862, Average: 0.014373
Train Epoch: 0 [44800/50000 (90%)]	Loss: 1.025594, Average: 0.016025

Train set: Average loss: 0.0190


Test set: Average loss: 0.0090, Accuracy: 7007/10000 (70%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.878075, Average: 0.013720
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.802323, Average: 0.012536
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.655698, Average: 0.010245
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.816413, Average: 0.012756
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.834883, Average: 0.013045

In [ ]:
plt.title("Resultats de l'entrenament")
plt.plot(range(1, (epochs + 1)), train_l,  c="red", label="train")
plt.plot(range(1,  (epochs + 1)), test_l,  c="green", label="test")
plt.legend();